# HMI to communicate with the created digital twin
This is a KWH HMI which allows to communicate with the digital twin, which was created and launched in the 08a notebook. All necessary modules are imported into the script, including the S³I library and ml40 reference implementation. This Notebook acts as an HMI in terms of the S³I and a client in terms of OAuth2 authentication.
Just go to the Cell drop-down menu and click on the Run All button.

In [1]:
import s3i
from ml.tools import find_broker_endpoint, make_config_file, load_config
from ml.dt_factory import create_dt_ref
from ml.fml40.features.properties.values.documents.jobs.felling_job import FellingJob
import json
import requests
import getpass
import uuid
from tools import print_with_timestamp

## Configure the notebook
In order to assign this notebook to your own HMI it needs a client id and the respective secret.In the next steps you should enter the id and the secret of your HMI in the following input fields.

In [2]:
print_with_timestamp("Assign a client to this notebook. (The id of your HMI)")
#hmi_id = input('[S3I]: Please enter your HMI id:').strip('," ')
#hmi_secret = getpass.getpass('[S3I]: Please enter your HMI secret:').strip('," ')
hmi_id = "s3i:b4f72a11-fb3d-47a3-b499-e7a88eaa5fe1"
hmi_secret = "84df6340-9d64-4ae6-9076-86c224ec484a"
print_with_timestamp("HMI id and secret are set.")

[S3I][2020-10-06 00:03:44]: Assign a client to this notebook. (The id of your HMI)
[S3I][2020-10-06 00:03:44]: HMI id and secret are set.


Next, you have to enter your username and password to authenticate yourself in S³I. With your entered username and password a hmi reference will be created.

In [3]:
print_with_timestamp("This is a KWH HMI, please log in!")
#username = input('[S3I]: Please enter your username:').strip('," ')
#password = getpass.getpass('[S3I]: Please enter the password:')
print_with_timestamp("Your username and password are set.")
username = "chen"
password = "8810515"

[S3I][2020-10-06 00:03:44]: This is a KWH HMI, please log in!
[S3I][2020-10-06 00:03:44]: Your username and password are set.


## Load the HMI JSON file 
A HMI reference can be created and launched using fml40 reference implementation library after the setting of the user credentials. All defined fml40 roles, functionalities and values etc. are respectively modelled according to the fml40 language. We have already prepared a JSON file that can be loaded to build a HMI reference in the next steps.

In [4]:
hmi_model = load_config('configs/{}'.format("config_my_hmi.json"))
hmi_ref = create_dt_ref(model=hmi_model, grant_type="password", secret=hmi_secret,
                        username=username, password=password, 
                        is_broker=True, is_repo=False, is_broker_rest=True)
hmi_proxy = hmi_ref.proxy()

## Run the HMI 
When the run_forever function is called, the HMI reference is started and meanwhile his listener is also started to receive the messages via S³I Broker. 

In [5]:
hmi_proxy.run_forever()

[S³I]: Launch my HMI
[S³I][IdP]: Connect with S3I-IdentityProvider


## Send a felling job to your digital twin
In the follow steps we will show you how to edit a S3I-B ServiceRequest in order to send a felling job to your digital twin, which was created in notebook 08a before. 

In [7]:
serv_req = s3i.messages.ServiceRequest()
receiver = input("[S³I]: Please enter the id of your digital twin: ")
hmi_endpoint = find_broker_endpoint(hmi_proxy.dir.get(), hmi_id)
service_type = input('[S3I]: What kind of service request would you like to choose? \n<AcceptsFellingJobs/acceptJob> \n<AcceptsFellingJobs/queryJobStatus> \n<AcceptsFellingJobs/removeJob>')
if "AcceptsFellingJobs/acceptJob" in service_type:
    job = FellingJob("", hmi_ref)
    parameters = {"job": job}
elif "AcceptsFellingJobs/queryJobStatus" in service_type:
    job_id = "id:1111"
    parameters={"id": job_id}
elif "AcceptsFellingJobs/removeJob" in service_type:
    job_id = "id:1111"
    parameters={"id": job_id}
else:
    print_with_timestamp("[S³I: Error in service type, please rewrite it!]")
    
serv_req.fillServiceRequest(
    senderUUID=hmi_id, receiverUUID=[receiver], sender_endpoint=hmi_endpoint,
    serviceType=service_type,
    parameters=parameters,
    msgUUID="s3i:{}".format(uuid.uuid4())
)
receiver_endpoint = find_broker_endpoint(hmi_proxy.dir.get(), thing_id=receiver)
resp = hmi_proxy.broker.get().send([receiver_endpoint], json.dumps(serv_req.msg))

[S³I]: Please enter the id of your digital twin: s3i:08ecc26f-2103-469c-a9d3-a1305c66ff26
{'sender': 's3i:08ecc26f-2103-469c-a9d3-a1305c66ff26', 'identifier': 's3i:7f389fce-dc62-4a45-996f-17aa991aafd7', 'receivers': ['s3i:b4f72a11-fb3d-47a3-b499-e7a88eaa5fe1'], 'messageType': 'serviceReply', 'serviceType': 'AcceptsFellingJobs/my_accept_job_func', 'replyingToMessage': 's3i:a11ea34b-dcab-4408-a90f-9ff0ccb9240d', 'results': {'ok': True}}
[S³I][Dir]: Connect with S3I-Directory
[S³I][Broker]: Connect with S3I-Broker
[S³I][Dir]: Connect with S3I-Directory
[S³I][Broker]: Connect with S3I-Broker
[S³I][Dir]: Connect with S3I-Directory
[S³I][Broker]: Connect with S3I-Broker
{'sender': 's3i:08ecc26f-2103-469c-a9d3-a1305c66ff26', 'identifier': 's3i:67482023-9abb-4f4d-b7aa-2858df26b22a', 'receivers': ['s3i:b4f72a11-fb3d-47a3-b499-e7a88eaa5fe1'], 'messageType': 'serviceReply', 'serviceType': 'AcceptsFellingJobs/my_accept_job_func', 'replyingToMessage': '6701c47c-cee7-4659-acd1-69fb57e56aa4', 'result